# Libraries

In [62]:
import numpy as np
import cmath
import math

# Task for 6-7

In [78]:
def simple_qr(matrix: np.array) -> np.array:
    q, r = np.linalg.qr(matrix)
    for _ in range(1_000):
        matrix = r @ q
        q, r = np.linalg.qr(matrix)
    return matrix

def get_roots(matrix: np.array) -> np.array:
    n = matrix.shape[0]
    approximated = np.vstack([simple_qr(matrix), np.zeros(n)])
    determinated = approximated.round(5)

    roots = np.zeros(n)
    i = 0
    while i < n:
        root = approximated[i][i]
        detr = determinated[i+1][i]
        if detr == 0:
            roots[i] = root
        else:
            # select block
            p = 1
            for j in range(i+1, n):
                if determinated[j+1][j] == 0:
                    break
                p += 1
            block = approximated[i:i+p+1, i:i+p+1]
            # p == 1 -> our target
            if p == 1:
                D = (block[1][1] + block[0][0])**2 - 4*(block[0][0]*block[1][1]-block[0][1]*block[1][0])
                # convert type
                if D < 0:
                    roots = roots.astype(complex)
                    l1 = (block[1][1] + block[0][0] - cmath.sqrt(D))/2
                    l2 = (block[1][1] + block[0][0] + cmath.sqrt(D))/2
                else:
                    l1 = (block[1][1] + block[0][0] - math.sqrt(D))/2
                    l2 = (block[1][1] + block[0][0] + math.sqrt(D))/2
                roots[i + p - 1] = l1
                roots[i + p - 0] = l2
            # p != 1 -> not our target
            else:
                __roots = np.linalg.eigvals(block)
                for j in range(p+1):
                    roots[i + j] = __roots[j]
            print(block)
            i += p
        i += 1
    return roots


In [88]:
n = 3
omega = 50

matrix = np.random.randint(-omega, omega, (n, n))
print(f"Initial matrix:\n{matrix}")
approximated_matrix = simple_qr(matrix)
print(f"Approximated matrix:\n{approximated_matrix.round(5)}")
roots = get_roots(matrix)
print(f"Function`s roots:\n{roots}")
eigen = np.linalg.eigvals(matrix)
print(f"True eigenvalues:\n{eigen}")

Initial matrix:
[[ 41 -33 -49]
 [-14   2  46]
 [ 24 -25  36]]
Approximated matrix:
[[ 56.25838  40.83786 -32.38817]
 [-57.29385  27.32262  -7.22314]
 [  0.        0.       -4.581  ]]
[[ 56.25838306  40.83786075]
 [-57.29384956  27.32261503]]
Function`s roots:
[41.79049905-46.1566743j 41.79049905+46.1566743j -4.58099809 +0.j       ]
True eigenvalues:
[41.79049905+46.1566743j 41.79049905-46.1566743j -4.58099809 +0.j       ]


# Task for 8

In [95]:
def hauseholder_qr(A):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        H[i:, i:] = make_householder(A[i:, i])
        Q = np.dot(Q, H)
        A = np.dot(H, A)
    return Q, A

def make_householder(a):
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    H = np.eye(a.shape[0])
    H -= (2 / np.dot(v, v)) * np.dot(v[:, None], v[None, :])
    return H

In [96]:
q_h, r_h = hauseholder_qr(matrix)
q, r = np.linalg.qr(matrix)

In [97]:
print(f'Householder\nQ:{q_h}\nR:{r_h}')
print(f'Numpy\nQ:{q}\nR:{r}')

Householder
Q:[[-0.82781841  0.01020003 -0.56090341]
 [ 0.2826697  -0.85605216 -0.43274998]
 [-0.48457663 -0.5167888   0.7057725 ]]
R:[[-4.95277700e+01  3.99977629e+01  3.61211498e+01]
 [-8.27939104e-16  1.08710149e+01 -5.84825973e+01]
 [ 1.57552734e-15  2.15124169e-17  3.29855779e+01]]
Numpy
Q:[[-0.82781841  0.01020003 -0.56090341]
 [ 0.2826697  -0.85605216 -0.43274998]
 [-0.48457663 -0.5167888   0.7057725 ]]
R:[[-49.52776999  39.99776288  36.12114982]
 [  0.          10.87101488 -58.4825973 ]
 [  0.           0.          32.98557789]]


In [101]:
q_h.round(5) == q.round(5), r_h.round(5) == r.round(5)

(array([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]),
 array([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))